# Vivino scraping
We use a mix of ```selenium``` to load the page and address the lazy loading and ```beautifulsoup``` to parse it.

In [152]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import re
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.common.exceptions import TimeoutException
import copy
from bs4 import BeautifulSoup
import time
import requests
import pandas as pd

# make def for the arguments search and vintage to scrape vivino
def scrape_vivino(search, vintage):
    
    # Create a new instance of the Firefox driver
    driver = webdriver.Firefox()

    base = "https://www.vivino.com/search/wines?q="
    html = base + search + "+" + vintage
    driver.get(html)
    driver.implicitly_wait(0.5) # gives an implicit wait for 20 seconds
    #try:
    #    button = WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="didomi-notice-agree-button"]')))
    #    button.click()
    #except NoSuchElementException:
    #    print("\tError finding xyz...")

    button = driver.find_element(By.XPATH,'/html/body/div[2]/section[1]/div/div/div/div[1]/div/div[1]/div/div[2]/div[1]/span[1]')
                                            
    button.click()
    driver.implicitly_wait(0.5) # gives an implicit wait for 20 seconds

    last_height = int(driver.execute_script("return document.body.scrollHeight"))

    #for i in range(1, total_height, 5):
    #    driver.execute_script("window.scrollTo(0, {});".format(i))
    # Get the scroll height


    # Scroll down step by step
    for i in range(1, last_height, 4):
        driver.execute_script(f"window.scrollTo(0, {i});")
        time.sleep(0.08)  # Adjust this value as per your internet speed
        last_height = int(driver.execute_script("return document.body.scrollHeight"))


    soup = BeautifulSoup(driver.page_source, 'html.parser')

    #save soup as html file
    #with open('vivino.html', 'w') as file:
    #    file.write(str(soup))
    # Close the driver
    driver.quit()

    # Find all span tags with the class 'vintage'
    vintage_spans = soup.find_all('span', class_='vintage')

    # Iterate over the span tags
    for span in vintage_spans:
        # Find the a tag with the class 'wine' within this span
        a_tag = span.find('a', class_='wine')
        
        # Extract the text from the a tag
        name = a_tag.text.strip()
        
        # Find the next sibling of the a tag and extract its text
        year = a_tag.next_sibling.strip()


    # Find all elements with the class 'bottleShot'
    bottle_shots = soup.find_all(class_='bottleShot')

    # Iterate over the elements with the class 'bottleShot'
    for bottle_shot in bottle_shots:
        # Find all source tags within this element
        source_tags = bottle_shot.find_all('source')
        
        # Iterate over the source tags
        for tag in source_tags:
            # Split the srcset attribute on comma and space to get the individual URLs
            urls = tag['srcset'].split(', ')
            
            
            # Split the URL on space to separate the URL and the resolution
            url, resolution = urls[1].split(' ')
            
            # Check if the resolution is '2x'
            if resolution == '2x':
                # Remove the "//" at the beginning of the URL
                final_url = 'https:' + url
                
                # Download the image
                response = requests.get(final_url, stream=True)
                if response.status_code == 200:
                    with open(f'{name} {year}.png', 'wb') as out_file:
                        out_file.write(response.content)

    # Find all span tags with the class 'headline'
    headline_spans = soup.find_all('span', class_='headline')


    # Iterate over the span tags
    for span in headline_spans:
        # Find the a tag with the class 'winery' within this span
        a_tag = span.find('a', class_='winery')
        
        # Extract the text from the a tag and add it to the list
        winery = a_tag.text.replace('\n', '')

    # Find the a tag with data-cy="breadcrumb-country"
    tag = soup.find('a', {'data-cy': 'breadcrumb-country'})

    # Get the text of the tag
    country = tag.text

    # Find the a tag with data-cy="breadcrumb-region"
    tag = soup.find('a', {'data-cy': 'breadcrumb-region'})

    # Get the text of the tag
    region = tag.text

    # Find the a tag with data-cy="breadcrumb-winetype"
    tag = soup.find('a', {'data-cy': 'breadcrumb-winetype'})

    # Get the text of the tag
    category = tag.text


    # Find the a tag with data-cy="breadcrumb-grape"
    tag = soup.find('a', {'data-cy': 'breadcrumb-grape'})

    # Get the text of the tag
    grape_type = tag.text


    # Find the h3 tag with class="wineTasteStyle-desktop__wineName--ML0zS"
    tag = soup.find('h3', {'class': 'wineTasteStyle-desktop__wineName--ML0zS'})

    # Get the text of the tag
    wine_style = tag.text


    # Find all the rows in the table
    rows = soup.select('.tasteStructure__tasteCharacteristic--jLtsE')
    percentages=[]
    # For each row
    for row in rows:
        # Find the cells
        cells = row.find_all('td')

        # The first cell contains the first property
        property1 = cells[0].text.strip()

        # The second cell contains the style attribute with the percentage
        style = cells[1].select_one('.indicatorBar__progress--3aXLX')['style']
        percentage = re.search(r'left:\s(\d+).\d+%;', style).group(1)

        # The third cell contains the second property
        property2 = cells[2].text.strip()
        # assign property1 and property2 to variables if not in list of Strings
        strings_to_avoid = ['Light', 'Bold', 'Dry', 'Sweet', 'Soft', "Acidic"]
        if property1 not in strings_to_avoid and property2 not in strings_to_avoid:
            specialCategory1 = property1
            specialCategory2 = property2
        
        # Print the properties and the percentage
        #print((int(percentage)+7.5)*0.01)
        #print(property1, property2)
        #append to percentages
        percentages.append((int(percentage)+7.5)*0.01)

    # Find all span tags with the class 'tasteNote__flavorGroup--1Uaen'
    flavor_group_spans = soup.find_all('span', class_='tasteNote__flavorGroup--1Uaen')

    # Extract the text from each span tag
    flavor_groups = [span.text.strip().capitalize() for span in flavor_group_spans]

    # Find all div tags with the class 'tasteNote__popularKeywords--1gIa2'
    popular_keywords_divs = soup.find_all('div', class_='tasteNote__popularKeywords--1gIa2')

    # Extract the text from each div tag, remove words that contain "..."
    popular_keywords = [' '.join(word.capitalize() for word in div.text.split() if "..." not in word) for div in popular_keywords_divs]

    popular_keywords = [keyword[:-1] if keyword.endswith(',') else keyword for keyword in popular_keywords]

    # Pair each flavor group with its corresponding popular keywords
    groupings = dict(zip(flavor_groups, popular_keywords))

    # Get only the first three groupings
    first_three_groupings = {k: groupings[k] for k in list(groupings)[:3]}

    # Format each key-value pair into a string and add it to the list
    main_impressions = [f"{k} ({v})" for k, v in first_three_groupings.items()]


    # Find all div tags with the class 'foodPairing__foodImage--2OYHg'
    food_divs = soup.find_all('div', class_='foodPairing__foodImage--2OYHg')

    # Extract the aria-label attribute from each div tag
    foods = [div.get('aria-label') for div in food_divs]
    foods = [re.sub(r'\(.*?\)', '', food) for food in foods]

    # Use join to convert the list to a string
    food_pairing = ', '.join(foods)
    # replace " , " with ", "
    food_pairing = food_pairing.replace(" , ", ", ")

    # Create a new instance of the Firefox driver
    driver = webdriver.Firefox()

    # Find the a tag
    tag = soup.find('a', {'class': 'anchor_anchor__m8Qi- wineTasteStyle-desktop__readMore--7NbVI anchor_vivinoLink__q1fW2'})

    # Get the href attribute of the tag
    href = tag.get('href')

    url = "https://www.vivino.com/"+href  # Outputs: /wine-styles/bordeaux-left-bank-pauillac
    driver.get(url)
    driver.implicitly_wait(1) # gives an implicit wait for 20 seconds

    sub_soup = BeautifulSoup(driver.page_source, 'html.parser')

    #save soup as html file
    #with open('sub_vivino.html', 'w') as file:
    #    file.write(str(sub_soup))
    # Close the driver
    driver.quit()


    # Find the div tag with class="wine-style-page__header__description__content"
    tag = sub_soup.find('div', {'class': 'wine-style-page__header__description__content'})

    # Get the text of the tag
    content = tag.text


    # Create a dictionary with the scraped data
    scraped_data = {
        'Name': [name], 
        'Picture ID': [name+" "+year], 
        'Year': [int(year)], 
        'Rating': [""],
        'Price Range': [""],
        'Winery': [winery],
        'Winery Description': [""],
        'Country': [country],
        'Region': [region],
        'Category': [category],
        'Light-to-Bold': [percentages[0]],
        'Cat1-to-Cat2': [percentages[1]],
        'Dry-to-Sweet': [percentages[2]],
        'Soft-to-Acidic': [percentages[3]],
        'SpecialCat1': [specialCategory1],
        'SpecialCat12': [specialCategory2],
        'Grapes': [grape_type],
        'Wine Style': [wine_style],
        'Wine Style Description': [content],
        'Alcohol': [""],
        'Main Impressions A': [main_impressions[0]],
        'Main Impressions B': [main_impressions[1]],
        'Main Impressions C': [main_impressions[2]],
        'Food Pairing': [food_pairing],
        'Notes': [""],
        'Color': [""],
        'New Line': [""]
    }
    return scraped_data

In [153]:
search = "Latour"
vintage = "2017"

scraped_data = scrape_vivino(search, vintage)

# Read the existing Excel file into a DataFrame
df = pd.read_excel('../Factsheet Automation/Wineportfolio Database.xlsx')

# Create a new DataFrame from the scraped data
new_data = pd.DataFrame(scraped_data)

# Concatenate the new data to the DataFrame
df = pd.concat([df, new_data], ignore_index=True)

# Write the DataFrame back to the Excel file
df.to_excel('test.xlsx', index=False)

In [145]:
# # Create a new instance of the Firefox driver
# driver = webdriver.Firefox()

# # Find the a tag
# tag = soup.find('a', {'class': 'winery'})

# # Get the href attribute of the tag
# href = tag.get('href')

# url = "https://www.vivino.com/"+href  # Outputs: /wine-styles/bordeaux-left-bank-pauillac
# driver.get(url)
# driver.implicitly_wait(1) # gives an implicit wait for 20 seconds

# sub_soup = BeautifulSoup(driver.page_source, 'html.parser')

# #save soup as html file
# with open('sub_vivino.html', 'w') as file:
#     file.write(str(sub_soup))
# # Close the driver
# driver.quit()